# Introduction

In the 21st century mostly every single process we do is automated. Starting from ordering our favourite food or requesting a taxi and ending with some advanced processes such as executing millions of millions transactions by simply pressing one button. And it all thanks to a major progress of Information Technologies.

Nowadays if we need to know where we can find some lovely dinner places in foreign country, what we need to do is simply open maps and by analyzing our geolocation the advanced machine can predict our preferences and provide us with most rated nearby restaurants in no time. But not that it is user-friendly and really reliable, it is, also, can be extremely precious for commercial use. For example, if we would get a project to build a new venue (office building, department store, grocery store, restaurant and so on.) in a city we have never been to, by using some machine learning algorithms we can cluster our whole city’s dataset and then visualize it on map to predict what would be the most efficient spot for a new start up. That is exactly what we will be doing today, imagine, that we got a business project to build a new office building in City of London the most advanced Borough in London and we need to find the best neighbourhood to proceed with. Please do take your seat ant let me take you through this fascinating journey where will be exploring and clustering different neighbourhoods in London to find which one best suits our needs.


# Data

First thing first in order for us to start analyze neighbourhoods in City of London we need to find dataset which one would include boroughs and neighbourhoods’ names and coordinates, because we will need them later on for data visualization. We will be using dataset provided by doogal.co.uk which is almost ideal for our project. Then we will just need to clean our data a little bit by dropping all insufficient columns and rename district to a borough and ward to neighbourhood columns, so our whole data would look more appealing and user-friendly.

Afterwards we will be calling foursquare api to find top 20 venues in every neighbourhood so we can cluster them and decide which one will be the most suitable for our project.


# Methodology

In this section first of all we will be cleaning our dataset, to prepare it for visualization and clustering.

Secondly when data is cleaned, we will be visualizing it on the map to see how it is distributed.

Thirdly we will be categorizing our dataset by using k-means clustering algorithm.

Fourthly we will be analyzing what cluster and what neighbourhood within a cluster is the nearest one to the station.

And lastly we will be transferring all of our processed dataset on the map, so that way we would be able to predict, what will be the most sufficient place for our project to begin.


### Installing and importing all required libraries for our project


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install folium -c conda-forge --yes
import folium # map rendering library
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.3.1-py_0        conda-forge
    folium:          

### Reading and examining size of our dataframe

In [3]:
df = pd.read_csv('https://www.doogal.co.uk/UKPostcodesCSV.ashx?area=London')
df.head(10)

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley South,0.218257,BR,BR1,Metropolitan Police,Thames Water,9F32C228+J5
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley North,0.253666,BR,BR1,Metropolitan Police,Thames Water,9F32C248+G3
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,1,1,2019-11-23,Bromley South,0.044559,BR,BR1,Metropolitan Police,NaN,9F32C228+2M
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley North,0.462939,BR,BR1,Metropolitan Police,Thames Water,9F32C237+RM
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley South,0.227664,BR,BR1,Metropolitan Police,Thames Water,9F32C227+HX
5,BR1 1AG,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley South,0.227664,BR,BR1,Metropolitan Police,Thames Water,9F32C227+HX
6,BR1 1AH,Yes,51.400441,0.017390,540432,168754,TQ404687,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley South,0.048906,BR,BR1,Metropolitan Police,Thames Water,9F32C228+5X
7,BR1 1AJ,Yes,51.400489,0.018833,540532,168762,TQ405687,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley South,0.115632,BR,BR1,Metropolitan Police,Thames Water,9F32C229+5G
8,BR1 1AL,Yes,51.406549,0.013130,540117,169425,TQ401694,Greater London,Bromley,Bromley Town,...,1,0,2019-11-23,Bromley North,0.332674,BR,BR1,Metropolitan Police,NaN,9F32C247+J7
9,BR1 1AX,No,51.408226,0.017578,540421,169620,TQ404696,Greater London,Bromley,Bromley Town,...,1,1,2019-11-23,Bromley North,0.042067,BR,BR1,Metropolitan Police,NaN,9F32C259+72


In [4]:
df.shape

(321375, 46)

### Cleaning our dataset and checking the size afterwards

In [5]:
# Dropping all unrequired collumns
df.drop(['In Use?', 'Easting', 'Northing', 'Grid Ref', 'County', 'District Code', 'Ward Code', 'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated', 'Parish', 
         'National Park', 'Population', 'Households', 'Built up area', 'Built up sub-division', 'Lower layer super output area', 'Rural/urban', 'Region', 'Altitude', 'London zone', 'LSOA Code',
         'Local authority', 'MSOA Code', 'Middle layer super output area', 'Parish Code', 'Census output area', 'Constituency Code', 'Index of Multiple Deprivation', 'Quality', 'User Type', 
         'Last updated', 'Postcode area', 'Postcode district', 'Police force', 'Water company', 'Plus Code'], axis=1)
df_lon = df[['District', 'Ward', 'Latitude', 'Longitude', 'Nearest station', 'Distance to station', 'Postcode']]
df_lon = df_lon.rename(columns={'District':'Borough', 'Ward':'Neighbourhood'})
df_lon.head(10)

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
0,Bromley,Bromley Town,51.401546,0.015415,Bromley South,0.218257,BR1 1AA
1,Bromley,Bromley Town,51.406333,0.015208,Bromley North,0.253666,BR1 1AB
2,Bromley,Bromley Town,51.400057,0.016715,Bromley South,0.044559,BR1 1AD
3,Bromley,Bromley Town,51.404543,0.014195,Bromley North,0.462939,BR1 1AE
4,Bromley,Bromley Town,51.401392,0.014948,Bromley South,0.227664,BR1 1AF
5,Bromley,Bromley Town,51.401392,0.014948,Bromley South,0.227664,BR1 1AG
6,Bromley,Bromley Town,51.400441,0.017390,Bromley South,0.048906,BR1 1AH
7,Bromley,Bromley Town,51.400489,0.018833,Bromley South,0.115632,BR1 1AJ
8,Bromley,Bromley Town,51.406549,0.013130,Bromley North,0.332674,BR1 1AL
9,Bromley,Bromley Town,51.408226,0.017578,Bromley North,0.042067,BR1 1AX


In [6]:
df_lon.shape

(321375, 7)

### Creating city of London dataset and finding the size

In [7]:
city_of_london_data = df_lon[df_lon['Borough'] == 'City of London'].reset_index(drop=True)

city_of_london_data.head(10)

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
0,City of London,Bishopsgate,51.518895,-0.078378,Liverpool Street,0.233547,E1 6AN
1,City of London,Portsoken,51.515567,-0.075635,Aldgate,0.136081,E1 7AA
2,City of London,Portsoken,51.515457,-0.076718,Aldgate,0.145028,E1 7AD
3,City of London,Portsoken,51.515613,-0.076899,Aldgate,0.166352,E1 7AE
4,City of London,Portsoken,51.515613,-0.076899,Aldgate,0.166352,E1 7AF
5,City of London,Portsoken,51.515630,-0.076279,Aldgate,0.150018,E1 7AW
6,City of London,Aldgate,51.515526,-0.078592,Aldgate,0.243597,E1 7AX
7,City of London,Aldgate,51.515526,-0.078592,Aldgate,0.243597,E1 7AY
8,City of London,Aldgate,51.515175,-0.077610,Aldgate,0.165426,E1 7BH
9,City of London,Portsoken,51.515432,-0.076806,Aldgate,0.145973,E1 7BS


In [8]:
city_of_london_data.shape

(6800, 7)

### Creating a new dictionary with unique neighbourhoods values

In [9]:
city_of_london_data_unique = {Neighbourhood: city_of_london_data[city_of_london_data['Neighbourhood'] == Neighbourhood] for Neighbourhood in city_of_london_data['Neighbourhood'].unique()}

### Converting dictionary in to a list and creating dataframe for each neighbourhood

In [10]:
dictlist =[]
for key, value in city_of_london_data_unique.items():
    temp = [key,value]
    dictlist.append(temp)

In [11]:
Bishopsgate = dictlist[0][1]
Bishopsgate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
0,City of London,Bishopsgate,51.518895,-0.078378,Liverpool Street,0.233547,E1 6AN
13,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DD
14,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DG
15,City of London,Bishopsgate,51.515791,-0.078869,Aldgate,0.275997,E1 7DJ
16,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DP


In [12]:
Portsoken = dictlist[1][1]
Portsoken.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
1,City of London,Portsoken,51.515567,-0.075635,Aldgate,0.136081,E1 7AA
2,City of London,Portsoken,51.515457,-0.076718,Aldgate,0.145028,E1 7AD
3,City of London,Portsoken,51.515613,-0.076899,Aldgate,0.166352,E1 7AE
4,City of London,Portsoken,51.515613,-0.076899,Aldgate,0.166352,E1 7AF
5,City of London,Portsoken,51.515630,-0.076279,Aldgate,0.150018,E1 7AW


In [13]:
Aldgate = dictlist[2][1]
Aldgate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
6,City of London,Aldgate,51.515526,-0.078592,Aldgate,0.243597,E1 7AX
7,City of London,Aldgate,51.515526,-0.078592,Aldgate,0.243597,E1 7AY
8,City of London,Aldgate,51.515175,-0.077610,Aldgate,0.165426,E1 7BH
10,City of London,Aldgate,51.514988,-0.077185,Aldgate,0.129444,E1 7BT
18,City of London,Aldgate,51.514882,-0.078905,Aldgate,0.234482,E1 7DS


In [14]:
Tower = dictlist[3][1]
Tower.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
60,City of London,Tower,51.511193,-0.073295,Tower Gateway,0.117704,E1 8AR
61,City of London,Tower,51.511193,-0.073295,Tower Gateway,0.117704,E1 8AS
62,City of London,Tower,51.511017,-0.073562,Tower Gateway,0.090839,E1 8AT
63,City of London,Tower,51.511646,-0.073521,Tower Gateway,0.152236,E1 8AW
64,City of London,Tower,51.511207,-0.074159,Tower Gateway,0.092140,E1 8BJ


In [15]:
Bread_Street = dictlist[4][1]
Bread_Street.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
82,City of London,Bread Street,51.515210,-0.099069,St. Pauls,0.111643,EC1A 1AE
212,City of London,Bread Street,51.515233,-0.100509,St. Pauls,0.207916,EC1A 7AR
214,City of London,Bread Street,51.515233,-0.100509,St. Pauls,0.207916,EC1A 7AT
215,City of London,Bread Street,51.515233,-0.100509,St. Pauls,0.207916,EC1A 7AU
216,City of London,Bread Street,51.515069,-0.098109,St. Pauls,0.045371,EC1A 7AW


In [16]:
Farringdon_Within = dictlist[5][1]
Farringdon_Within.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
83,City of London,Farringdon Within,51.517883,-0.097516,Barbican,0.259553,EC1A 1AL
84,City of London,Farringdon Within,51.516359,-0.098906,St. Pauls,0.192845,EC1A 1HQ
85,City of London,Farringdon Within,51.516359,-0.098906,St. Pauls,0.192845,EC1A 1LP
86,City of London,Farringdon Within,51.516109,-0.099031,St. Pauls,0.173852,EC1A 1ZZ
87,City of London,Farringdon Within,51.516273,-0.102498,City Thameslink,0.270586,EC1A 2AA


In [17]:
Farringdon_Without = dictlist[6][1]
Farringdon_Without.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
92,City of London,Farringdon Without,51.517473,-0.103327,Farringdon,0.320191,EC1A 2AL
98,City of London,Farringdon Without,51.517187,-0.102878,Farringdon,0.360882,EC1A 2AY
106,City of London,Farringdon Without,51.516996,-0.102727,City Thameslink,0.345373,EC1A 2BS
110,City of London,Farringdon Without,51.516842,-0.102676,City Thameslink,0.329190,EC1A 2DH
111,City of London,Farringdon Without,51.516996,-0.102727,City Thameslink,0.345373,EC1A 2DJ


In [18]:
Castle_Baynard = dictlist[7][1]
Castle_Baynard.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
93,City of London,Castle Baynard,51.517147,-0.107636,Chancery Lane,0.334490,EC1A 2AP
105,City of London,Castle Baynard,51.517147,-0.107636,Chancery Lane,0.334490,EC1A 2BQ
107,City of London,Castle Baynard,51.516727,-0.107783,Chancery Lane,0.349365,EC1A 2DA
126,City of London,Castle Baynard,51.517147,-0.107636,Chancery Lane,0.334490,EC1A 2ES
127,City of London,Castle Baynard,51.517147,-0.107636,Chancery Lane,0.334490,EC1A 2ET


In [19]:
Cheap = dictlist[8][1]
Cheap.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
139,City of London,Cheap,51.515226,-0.096733,St. Pauls,0.072408,EC1A 4AA
141,City of London,Cheap,51.515226,-0.096733,St. Pauls,0.072408,EC1A 4AD
144,City of London,Cheap,51.516085,-0.097591,St. Pauls,0.138874,EC1A 4AQ
145,City of London,Cheap,51.516411,-0.097721,St. Pauls,0.175399,EC1A 4AS
146,City of London,Cheap,51.516411,-0.097721,St. Pauls,0.175399,EC1A 4BB


In [20]:
Aldersgate = dictlist[9][1]
Aldersgate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
142,City of London,Aldersgate,51.521025,-0.097212,Barbican,0.096628,EC1A 4AJ
167,City of London,Aldersgate,51.516554,-0.096562,St. Pauls,0.203328,EC1A 4ER
169,City of London,Aldersgate,51.516984,-0.097020,St. Pauls,0.241785,EC1A 4EU
171,City of London,Aldersgate,51.517469,-0.097533,St. Pauls,0.292669,EC1A 4HD
173,City of London,Aldersgate,51.517029,-0.096442,St. Pauls,0.255961,EC1A 4HJ


In [21]:
Cripplegate = dictlist[10][1]
Cripplegate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
481,City of London,Cripplegate,51.522476,-0.097411,Barbican,0.252127,EC1M 7AA
553,City of London,Cripplegate,51.521580,-0.095358,Barbican,0.222944,EC1Y 0AA
554,City of London,Cripplegate,51.522513,-0.095262,Barbican,0.306773,EC1Y 0HA
555,City of London,Cripplegate,51.521813,-0.095911,Barbican,0.217236,EC1Y 0RB
556,City of London,Cripplegate,51.522121,-0.095999,Barbican,0.242959,EC1Y 0RD


In [22]:
Coleman_Street = dictlist[11][1]
Coleman_Street.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
579,City of London,Coleman Street,51.520686,-0.090178,Moorgate,0.258532,EC1Y 4AG
580,City of London,Coleman Street,51.520794,-0.090706,Moorgate,0.283560,EC1Y 4SA
581,City of London,Coleman Street,51.520728,-0.092208,Moorgate,0.335863,EC1Y 4SD
582,City of London,Coleman Street,51.520485,-0.091642,Moorgate,0.289769,EC1Y 4SJ
583,City of London,Coleman Street,51.520485,-0.091642,Moorgate,0.289769,EC1Y 4SL


In [23]:
Broad_Street = dictlist[12][1]
Broad_Street.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
651,City of London,Broad Street,51.516838,-0.084732,Liverpool Street,0.191314,EC2M 1BB
652,City of London,Broad Street,51.515874,-0.084628,Liverpool Street,0.252504,EC2M 1DL
654,City of London,Broad Street,51.515874,-0.084628,Liverpool Street,0.252504,EC2M 1JA
655,City of London,Broad Street,51.516686,-0.083559,Liverpool Street,0.135918,EC2M 1JB
656,City of London,Broad Street,51.517078,-0.084422,Liverpool Street,0.160955,EC2M 1JD


In [24]:
Cornhill = dictlist[13][1]
Cornhill.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
716,City of London,Cornhill,51.515851,-0.083188,Liverpool Street,0.204712,EC2M 1RJ
717,City of London,Cornhill,51.515851,-0.083188,Liverpool Street,0.204712,EC2M 1RL
718,City of London,Cornhill,51.515851,-0.083188,Liverpool Street,0.204712,EC2M 1RN
804,City of London,Cornhill,51.515940,-0.082452,Liverpool Street,0.184797,EC2M 3AL
811,City of London,Cornhill,51.515851,-0.083188,Liverpool Street,0.204712,EC2M 3TA


In [25]:
Walbrook = dictlist[14][1]
Walbrook.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
1339,City of London,Walbrook,51.514541,-0.086140,Bank,0.238160,EC2N 1AR
1385,City of London,Walbrook,51.514124,-0.087584,Bank,0.130551,EC2N 1EY
1392,City of London,Walbrook,51.514532,-0.086644,Bank,0.209485,EC2N 1HH
1396,City of London,Walbrook,51.514532,-0.086644,Bank,0.209485,EC2N 1HP
1398,City of London,Walbrook,51.514532,-0.086644,Bank,0.209485,EC2N 1HR


In [26]:
Lime_Street = dictlist[15][1]
Lime_Street.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
1533,City of London,Lime Street,51.514054,-0.083263,Liverpool Street,0.400024,EC2N 4AE
1534,City of London,Lime Street,51.514694,-0.082833,Liverpool Street,0.325233,EC2N 4AF
1536,City of London,Lime Street,51.514936,-0.082779,Liverpool Street,0.298087,EC2N 4AH
1537,City of London,Lime Street,51.515033,-0.082660,Liverpool Street,0.286461,EC2N 4AJ
1541,City of London,Lime Street,51.514054,-0.083263,Liverpool Street,0.400024,EC2N 4AQ


In [27]:
Cordwainer = dictlist[16][1]
Cordwainer.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
1569,City of London,Cordwainer,51.513272,-0.090502,Bank,0.104195,EC2P 2AH
1572,City of London,Cordwainer,51.513824,-0.093448,Mansion House,0.199347,EC2P 2BA
1815,City of London,Cordwainer,51.513296,-0.091942,Bank,0.203520,EC2R 8AT
1830,City of London,Cordwainer,51.513272,-0.090502,Bank,0.104195,EC2R 8BS
1836,City of London,Cordwainer,51.513296,-0.091942,Bank,0.203520,EC2R 8DH


In [28]:
Bassishaw = dictlist[17][1]
Bassishaw.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
1570,City of London,Bassishaw,51.517168,-0.091146,Moorgate,0.203829,EC2P 2AJ
1571,City of London,Bassishaw,51.517168,-0.091146,Moorgate,0.203829,EC2P 2AT
1576,City of London,Bassishaw,51.517168,-0.091146,Moorgate,0.203829,EC2P 2DY
1578,City of London,Bassishaw,51.516015,-0.093270,St. Pauls,0.325024,EC2P 2NA
1579,City of London,Bassishaw,51.516671,-0.093776,St. Pauls,0.332379,EC2P 2NQ


In [29]:
Langbourn = dictlist[18][1]
Langbourn.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
2561,City of London,Langbourn,51.512209,-0.080458,Fenchurch Street,0.124881,EC3A 2AB
2568,City of London,Langbourn,51.512209,-0.080458,Fenchurch Street,0.124881,EC3A 2AY
3106,City of London,Langbourn,51.512233,-0.081898,Fenchurch Street,0.217619,EC3M 2RB
3107,City of London,Langbourn,51.512209,-0.080458,Fenchurch Street,0.124881,EC3M 2RD
3108,City of London,Langbourn,51.512209,-0.080458,Fenchurch Street,0.124881,EC3M 2RH


In [30]:
Bridge = dictlist[19][1]
Bridge.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
3022,City of London,Bridge,51.510603,-0.085599,Monument,0.040869,EC3M 1AA
3024,City of London,Bridge,51.510483,-0.084854,Monument,0.093721,EC3M 1AD
3025,City of London,Bridge,51.510606,-0.085253,Monument,0.064741,EC3M 1AE
3026,City of London,Bridge,51.510815,-0.085388,Monument,0.059026,EC3M 1AG
3027,City of London,Bridge,51.510578,-0.085210,Monument,0.067908,EC3M 1AH


In [31]:
Candlewick = dictlist[20][1]
Candlewick.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
3023,City of London,Candlewick,51.510680,-0.085956,Monument,0.016985,EC3M 1AB
3040,City of London,Candlewick,51.510506,-0.086294,Monument,0.015638,EC3M 1BJ
3041,City of London,Candlewick,51.510506,-0.086294,Monument,0.015638,EC3M 1BL
3160,City of London,Candlewick,51.511405,-0.086257,Monument,0.086203,EC3M 3DU
3877,City of London,Candlewick,51.510506,-0.086294,Monument,0.015638,EC3R 6BP


In [32]:
Billingsgate = dictlist[21][1]
Billingsgate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
3029,City of London,Billingsgate,51.510437,-0.083718,Monument,0.172188,EC3M 1AL
3030,City of London,Billingsgate,51.510437,-0.083718,Monument,0.172188,EC3M 1AN
3031,City of London,Billingsgate,51.509560,-0.083452,Monument,0.223516,EC3M 1AP
3037,City of London,Billingsgate,51.510459,-0.083414,Monument,0.192809,EC3M 1BB
3055,City of London,Billingsgate,51.509560,-0.083452,Monument,0.223516,EC3M 1DY


In [33]:
Vintry = dictlist[22][1]
Vintry.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
4964,City of London,Vintry,51.512467,-0.09630,Mansion House,0.151249,EC4M 5SA
4966,City of London,Vintry,51.512444,-0.09486,Mansion House,0.060251,EC4M 5SD
4969,City of London,Vintry,51.512444,-0.09486,Mansion House,0.060251,EC4M 5SJ
4970,City of London,Vintry,51.512444,-0.09486,Mansion House,0.060251,EC4M 5SL
4971,City of London,Vintry,51.512444,-0.09486,Mansion House,0.060251,EC4M 5SN


In [34]:
Queenhithe = dictlist[23][1]
Queenhithe.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
4978,City of London,Queenhithe,51.511545,-0.094898,Mansion House,0.077697,EC4M 6GB
4987,City of London,Queenhithe,51.511545,-0.094898,Mansion House,0.077697,EC4M 6XN
5337,City of London,Queenhithe,51.511545,-0.094898,Mansion House,0.077697,EC4N 4TX
5341,City of London,Queenhithe,51.511545,-0.094898,Mansion House,0.077697,EC4N 4UB
5444,City of London,Queenhithe,51.511545,-0.094898,Mansion House,0.077697,EC4N 6LT


In [35]:
Dowgate = dictlist[24][1]
Dowgate.head()

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
5359,City of London,Dowgate,51.511475,-0.090577,Cannon Street,0.022259,EC4N 5AH
5361,City of London,Dowgate,51.511475,-0.090577,Cannon Street,0.022259,EC4N 5AL
5362,City of London,Dowgate,51.511475,-0.090577,Cannon Street,0.022259,EC4N 5AN
5364,City of London,Dowgate,51.511475,-0.090577,Cannon Street,0.022259,EC4N 5AQ
5371,City of London,Dowgate,51.511475,-0.090577,Cannon Street,0.022259,EC4N 5AY


### Taking 20 samples from each neighbourhood and combining them into a new dataframe

In [36]:
Bishopsgate_samples = Bishopsgate.head(20)

In [37]:
Portsoken_samples = Portsoken.head(20)

In [38]:
Aldgate_samples = Aldgate.head(20)

In [39]:
Tower_samples = Tower.head(20)

In [40]:
Bread_Street_samples = Bread_Street.head(20)

In [41]:
Farringdon_Within_samples = Farringdon_Within.head(20)

In [42]:
Farringdon_Without_samples = Farringdon_Without.head(20)

In [43]:
Castle_Baynard_samples = Castle_Baynard.head(20)

In [44]:
Cheap_samples = Cheap.head(20)

In [45]:
Aldersgate_samples = Aldersgate.head(20)

In [46]:
Cripplegate_samples = Cripplegate.head(20)

In [47]:
Coleman_Street_samples = Coleman_Street.head(20)

In [48]:
Broad_Street_samples = Broad_Street.head(20)

In [49]:
Cornhill_samples = Cornhill.head(20)

In [50]:
Walbrook_samples = Walbrook.head(20)

In [51]:
Lime_Street_samples = Lime_Street.head(20)

In [52]:
Cordwainer_samples = Cordwainer.head(20)

In [53]:
Bassishaw_samples = Bassishaw.head(20)

In [54]:
Langbourn_samples = Langbourn.head(20)

In [55]:
Bridge_samples = Bridge.head(20)

In [56]:
Candlewick_samples = Candlewick.head(20)

In [57]:
Billingsgate_samples = Billingsgate.head(20)

In [58]:
Vintry_samples = Vintry.head(20)

In [59]:
Queenhithe_samples = Queenhithe.head(20)

In [60]:
Dowgate_samples = Dowgate.head(20)

In [61]:
# Combining all samples in one dataframe
frames = [Bishopsgate_samples, Portsoken_samples, Aldgate_samples, Tower_samples, Bread_Street_samples,
Farringdon_Within_samples, Farringdon_Without_samples, Castle_Baynard_samples, Cheap_samples, Aldersgate_samples,
Cripplegate_samples, Coleman_Street_samples, Broad_Street_samples, Cornhill_samples, Walbrook_samples, Lime_Street_samples, Cordwainer_samples,
Bassishaw_samples, Langbourn_samples, Bridge_samples, Candlewick_samples, Billingsgate_samples, Vintry_samples, Queenhithe_samples, Dowgate_samples]
city_of_london_data_samples = pd.concat(frames)
city_of_london_data_samples

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode
0,City of London,Bishopsgate,51.518895,-0.078378,Liverpool Street,0.233547,E1 6AN
13,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DD
14,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DG
15,City of London,Bishopsgate,51.515791,-0.078869,Aldgate,0.275997,E1 7DJ
16,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DP
17,City of London,Bishopsgate,51.515755,-0.078998,Aldgate,0.281037,E1 7DR
19,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DW
27,City of London,Bishopsgate,51.517970,-0.078054,Liverpool Street,0.233174,E1 7ES
29,City of London,Bishopsgate,51.518576,-0.078274,Liverpool Street,0.227487,E1 7HA
30,City of London,Bishopsgate,51.516655,-0.077389,Aldgate,0.284403,E1 7HE


### Finding latitude and longitude of the City of London

In [62]:
address = 'City of London'

geolocator = Nominatim(user_agent="lon_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of City of London are {}, {}.'.format(latitude, longitude))

The geographical coordinate of City of London are 51.5156177, -0.0919983.


In [63]:
city_of_london_map = folium.Map(location=[latitude, longitude], zoom_start=14.5)

# add markers to map
for lat, lng, label in zip(city_of_london_data_samples['Latitude'], city_of_london_data_samples['Longitude'], city_of_london_data_samples['Neighbourhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='purple',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(city_of_london_map)
    
city_of_london_map

### Defining foursquare credentials and version

In [64]:
# @hidden_cell
CLIENT_ID = 'P3UQT5BGLDRLIOQJ3HP32HG3RMPAJ5KPVUKVHO4BAWIA4ZWF' # your Foursquare ID
CLIENT_SECRET = 'PK5TD1CDMDPW1CIQF5BLSLISM0SVVSBO2TJCIVNGBLGVNXZI' # your Foursquare Secret.
VERSION = '20180605' # Foursquare API version

### Creating city_of_london_venues dataframe by calling foursquare api and finding 100 venues for each neighbourhood within 500 meters

In [65]:
%%capture

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues);
    
city_of_london_venues = getNearbyVenues(names=city_of_london_data_samples['Neighbourhood'],
                                   latitudes=city_of_london_data_samples['Latitude'],
                                   longitudes=city_of_london_data_samples['Longitude']
                                  )

### Examining the size and appearance of the dataframe

In [66]:
city_of_london_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bishopsgate,51.518895,-0.078378,Kastner & Ovens,51.517913,-0.076465,Café
1,Bishopsgate,51.518895,-0.078378,Ottolenghi,51.518272,-0.077177,Mediterranean Restaurant
2,Bishopsgate,51.518895,-0.078378,Pizza Union,51.517699,-0.077416,Pizza Place
3,Bishopsgate,51.518895,-0.078378,Old Spitalfields Market,51.519668,-0.075375,Flea Market
4,Bishopsgate,51.518895,-0.078378,Honest Burgers,51.518042,-0.077957,Burger Joint
5,Bishopsgate,51.518895,-0.078378,The Mayor of Scaredy Cat Town,51.518524,-0.078882,Speakeasy
6,Bishopsgate,51.518895,-0.078378,The Breakfast Club,51.518386,-0.078784,Breakfast Spot
7,Bishopsgate,51.518895,-0.078378,New Street Wine,51.517335,-0.079857,Wine Shop
8,Bishopsgate,51.518895,-0.078378,The Williams Ale & Cider House,51.518517,-0.078447,Pub
9,Bishopsgate,51.518895,-0.078378,Galvin La Chapelle,51.520292,-0.078179,French Restaurant


In [67]:
city_of_london_venues.shape

(48363, 7)

### Checking how many venues were returned for each neighbourhood

In [68]:
city_of_london_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Aldersgate,1887,1887,1887,1887,1887,1887
Aldgate,2000,2000,2000,2000,2000,2000
Bassishaw,1970,1970,1970,1970,1970,1970
Billingsgate,1806,1806,1806,1806,1806,1806
Bishopsgate,2000,2000,2000,2000,2000,2000
Bread Street,2000,2000,2000,2000,2000,2000
Bridge,1998,1998,1998,1998,1998,1998
Broad Street,2000,2000,2000,2000,2000,2000
Candlewick,1933,1933,1933,1933,1933,1933


### Finding out how many unique categories can be curated from all the returned venues

In [69]:
print('There are {} uniques categories.'.format(len(city_of_london_venues['Venue Category'].unique())))

There are 166 uniques categories.


### Analyzing occurrence of each venue in neighbourhood with onehot encoding

In [71]:
city_of_london_onehot = pd.get_dummies(city_of_london_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighbourhood column back to dataframe
city_of_london_onehot['Neighbourhood'] = city_of_london_venues['Neighbourhood'] 

# moving neighbourhood column to the first columncity_of_london_onehot
fixed_columns = [city_of_london_onehot.columns[-1]] + list(city_of_london_onehot.columns[:-1])
city_of_london_onehot = city_of_london_onehot[fixed_columns]

city_of_london_onehot.head(10)

,Neighbourhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Bishopsgate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Examining size of a newly created dataframe

In [72]:
city_of_london_onehot.shape

(48363, 167)

### Grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each category


In [73]:
city_of_london_grouped = city_of_london_onehot.groupby('Neighbourhood').mean().reset_index()
city_of_london_grouped

,Neighbourhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Aldersgate,0.003180,0.0000,0.000000,0.026497,0.000000,0.000000,0.005299,0.004769,0.005299,...,0.010069,0.000000,0.000000,0.015898,0.000000,0.015898,0.0000,0.0000,0.010069,0.001590
1,Aldgate,0.000000,0.0000,0.012000,0.002000,0.000000,0.000000,0.017500,0.010000,0.000000,...,0.018000,0.000000,0.000000,0.005000,0.000000,0.015000,0.0100,0.0000,0.002000,0.000000
2,Bassishaw,0.000000,0.0000,0.000508,0.030457,0.000000,0.000000,0.013706,0.000000,0.000000,...,0.004569,0.004061,0.003046,0.017259,0.000000,0.013198,0.0000,0.0000,0.004061,0.020305
3,Billingsgate,0.000000,0.0000,0.011074,0.000000,0.000000,0.000000,0.033223,0.011074,0.000000,...,0.011074,0.000000,0.000000,0.000000,0.000000,0.012735,0.0000,0.0000,0.000000,0.000000
4,Bishopsgate,0.000000,0.0000,0.010000,0.010000,0.000000,0.000000,0.002500,0.010000,0.000000,...,0.006000,0.000000,0.000000,0.019500,0.000000,0.016000,0.0145,0.0005,0.010000,0.000000
5,Bread Street,0.010000,0.0000,0.000000,0.004500,0.000000,0.000000,0.017500,0.000000,0.000000,...,0.000000,0.002500,0.001000,0.029000,0.000000,0.031000,0.0000,0.0000,0.007500,0.002000
6,Bridge,0.000000,0.0000,0.010010,0.000000,0.000000,0.000000,0.043544,0.010010,0.000000,...,0.010010,0.002503,0.000000,0.000000,0.000000,0.010010,0.0000,0.0000,0.000000,0.000000
7,Broad Street,0.000000,0.0000,0.016000,0.000000,0.000000,0.000000,0.016500,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.009000,0.002500,0.010500,0.0110,0.0000,0.000500,0.019000
8,Candlewick,0.000000,0.0000,0.010347,0.000517,0.000000,0.000000,0.034144,0.001035,0.000000,...,0.010347,0.010347,0.000000,0.001552,0.000000,0.010347,0.0000,0.0000,0.000000,0.004139
9,Castle Baynard,0.000500,0.0005,0.009500,0.001000,0.000000,0.000000,0.012500,0.000000,0.000000,...,0.000000,0.000500,0.000500,0.011000,0.009000,0.030000,0.0000,0.0000,0.001000,0.000000


### Creating dataframe for each neighbourhood along with top 20 venues

In [74]:
num_top_venues = 20

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = city_of_london_grouped['Neighbourhood']

for ind in np.arange(city_of_london_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_of_london_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Aldersgate,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,Plaza,Art Gallery,French Restaurant,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
1,Aldgate,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,English Restaurant,Salad Place,Garden,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2,Bassishaw,Coffee Shop,Italian Restaurant,Hotel,Gym / Fitness Center,Sandwich Place,Art Gallery,Sushi Restaurant,Seafood Restaurant,Restaurant,...,French Restaurant,Steakhouse,Clothing Store,Yoga Studio,Concert Hall,Vietnamese Restaurant,Roof Deck,Indian Restaurant,Plaza,Pub
3,Billingsgate,Hotel,Coffee Shop,Gym / Fitness Center,Pub,Restaurant,French Restaurant,Garden,Asian Restaurant,English Restaurant,...,Café,Sandwich Place,Salad Place,Historic Site,Scenic Lookout,Hotel Bar,Burger Joint,Fast Food Restaurant,Italian Restaurant,Seafood Restaurant
4,Bishopsgate,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,Burger Joint,Chinese Restaurant,Restaurant,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
5,Bread Street,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,Vietnamese Restaurant,Japanese Restaurant,Burger Joint,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
6,Bridge,Coffee Shop,Hotel,Gym / Fitness Center,Pub,Restaurant,Asian Restaurant,French Restaurant,Burger Joint,Historic Site,...,Italian Restaurant,Garden,English Restaurant,Cocktail Bar,Salad Place,Café,Fast Food Restaurant,Bar,Seafood Restaurant,History Museum
7,Broad Street,Coffee Shop,Pub,Restaurant,Hotel,Gym / Fitness Center,Cocktail Bar,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,...,Food Truck,Salad Place,English Restaurant,Event Space,Breakfast Spot,Boxing Gym,Burger Joint,Plaza,Yoga Studio,Historic Site
8,Candlewick,Coffee Shop,Gym / Fitness Center,Pub,Italian Restaurant,Hotel,Restaurant,Asian Restaurant,Bar,Burger Joint,...,Historic Site,English Restaurant,Garden,Seafood Restaurant,Indian Restaurant,Fast Food Restaurant,Steakhouse,Salad Place,Sandwich Place,Café
9,Castle Baynard,Coffee Shop,Pub,Sandwich Place,Italian Restaurant,Wine Bar,Gym / Fitness Center,French Restaurant,Burrito Place,Beer Bar,...,Falafel Restaurant,Japanese Restaurant,Bar,Fast Food Restaurant,Salad Place,Deli / Bodega,Tea Room,Café,Asian Restaurant,Steakhouse


### Clustering neighbourhoods

In [75]:
kclusters = 5

city_of_london_grouped_clustering = city_of_london_grouped.drop('Neighbourhood', 1)

# runing k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=4, n_init=12).fit(city_of_london_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:25]

array([0, 3, 1, 3, 4, 2, 3, 4, 3, 2, 2, 0, 1, 4, 0, 1, 2, 2, 3, 4, 4, 1,
       3, 1, 1], dtype=int32)

### Creating a new dataframe with clusters and top 20 venues for each neighborhood

In [76]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_of_london_merged = city_of_london_data_samples

# merging city_of_london_grouped with city_of_london_data to add latitude/longitude for each neighbourhood
city_of_london_merged = city_of_london_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

city_of_london_merged.head(10)

,Borough,Neighbourhood,Latitude,Longitude,Nearest station,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,City of London,Bishopsgate,51.518895,-0.078378,Liverpool Street,0.233547,E1 6AN,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
13,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DD,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
14,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DG,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
15,City of London,Bishopsgate,51.515791,-0.078869,Aldgate,0.275997,E1 7DJ,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
16,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DP,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
17,City of London,Bishopsgate,51.515755,-0.078998,Aldgate,0.281037,E1 7DR,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
19,City of London,Bishopsgate,51.515780,-0.078867,Aldgate,0.275173,E1 7DW,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
27,City of London,Bishopsgate,51.517970,-0.078054,Liverpool Street,0.233174,E1 7ES,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
29,City of London,Bishopsgate,51.518576,-0.078274,Liverpool Street,0.227487,E1 7HA,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
30,City of London,Bishopsgate,51.516655,-0.077389,Aldgate,0.284403,E1 7HE,4,Coffee Shop,Pub,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store


### Examining the size of the dataset

In [77]:
city_of_london_merged.shape

(500, 28)

### Examining Clusters

#### Cluster 0

In [78]:
city_of_london_merged.loc[city_of_london_merged['Cluster Labels'] == 0, city_of_london_merged.columns[[1] + list(range(5, city_of_london_merged.shape[1]))]]

,Neighbourhood,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
142,Aldersgate,0.096628,EC1A 4AJ,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
167,Aldersgate,0.203328,EC1A 4ER,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
169,Aldersgate,0.241785,EC1A 4EU,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
171,Aldersgate,0.292669,EC1A 4HD,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
173,Aldersgate,0.255961,EC1A 4HJ,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
182,Aldersgate,0.039490,EC1A 4JP,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
183,Aldersgate,0.067933,EC1A 4JR,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
184,Aldersgate,0.126671,EC1A 4LA,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
193,Aldersgate,0.078380,EC1A 4LX,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater
231,Aldersgate,0.238767,EC1A 7BQ,0,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Sandwich Place,Food Truck,...,Cocktail Bar,Concert Hall,Hotel,Park,Modern European Restaurant,Scenic Lookout,Pub,Garden,Burger Joint,Indie Movie Theater


#### Cluster 1

In [79]:
city_of_london_merged.loc[city_of_london_merged['Cluster Labels'] == 1, city_of_london_merged.columns[[1] + list(range(5, city_of_london_merged.shape[1]))]]

,Neighbourhood,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1339,Walbrook,0.238160,EC2N 1AR,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1385,Walbrook,0.130551,EC2N 1EY,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1392,Walbrook,0.209485,EC2N 1HH,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1396,Walbrook,0.209485,EC2N 1HP,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1398,Walbrook,0.209485,EC2N 1HR,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1408,Walbrook,0.209485,EC2N 1LA,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1413,Walbrook,0.209485,EC2N 1TE,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1415,Walbrook,0.168627,EC2N 2AB,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1416,Walbrook,0.226503,EC2N 2AD,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant
1424,Walbrook,0.130551,EC2N 2AR,1,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Hotel,Restaurant,Café,...,Cocktail Bar,Burger Joint,Salad Place,Modern European Restaurant,Yoga Studio,Event Space,Sushi Restaurant,Wine Bar,English Restaurant,Seafood Restaurant


#### Cluster 2

In [80]:
city_of_london_merged.loc[city_of_london_merged['Cluster Labels'] == 2, city_of_london_merged.columns[[1] + list(range(5, city_of_london_merged.shape[1]))]]

,Neighbourhood,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
82,Bread Street,0.111643,EC1A 1AE,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
212,Bread Street,0.207916,EC1A 7AR,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
214,Bread Street,0.207916,EC1A 7AT,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
215,Bread Street,0.207916,EC1A 7AU,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
216,Bread Street,0.045371,EC1A 7AW,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
217,Bread Street,0.207916,EC1A 7AX,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
218,Bread Street,0.111643,EC1A 7AY,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
219,Bread Street,0.190272,EC1A 7AZ,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
233,Bread Street,0.207916,EC1A 7BS,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery
263,Bread Street,0.190272,EC1A 7HD,2,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Pub,Sandwich Place,Wine Bar,...,Scenic Lookout,French Restaurant,Modern European Restaurant,Plaza,Café,Park,Grocery Store,Asian Restaurant,English Restaurant,Bakery


#### Cluster 3

In [81]:
city_of_london_merged.loc[city_of_london_merged['Cluster Labels'] == 3, city_of_london_merged.columns[[1] + list(range(5, city_of_london_merged.shape[1]))]]

,Neighbourhood,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
6,Aldgate,0.243597,E1 7AX,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
7,Aldgate,0.243597,E1 7AY,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
8,Aldgate,0.165426,E1 7BH,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
10,Aldgate,0.129444,E1 7BT,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
18,Aldgate,0.234482,E1 7DS,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2543,Aldgate,0.252820,EC3A 1AA,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2544,Aldgate,0.256556,EC3A 1AB,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2545,Aldgate,0.263277,EC3A 1AH,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2546,Aldgate,0.263277,EC3A 1AJ,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant
2547,Aldgate,0.263277,EC3A 1AL,3,Hotel,Coffee Shop,Cocktail Bar,Gym / Fitness Center,Restaurant,Pub,...,French Restaurant,Turkish Restaurant,Asian Restaurant,Beer Bar,Steakhouse,Indian Restaurant,Café,Wine Bar,Event Space,Sushi Restaurant


#### Cluster 4

In [82]:
city_of_london_merged.loc[city_of_london_merged['Cluster Labels'] == 4, city_of_london_merged.columns[[1] + list(range(5, city_of_london_merged.shape[1]))]]

,Neighbourhood,Distance to station,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Bishopsgate,0.233547,E1 6AN,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
13,Bishopsgate,0.275173,E1 7DD,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
14,Bishopsgate,0.275173,E1 7DG,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
15,Bishopsgate,0.275997,E1 7DJ,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
16,Bishopsgate,0.275173,E1 7DP,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
17,Bishopsgate,0.281037,E1 7DR,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
19,Bishopsgate,0.275173,E1 7DW,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
27,Bishopsgate,0.233174,E1 7ES,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
29,Bishopsgate,0.227487,E1 7HA,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store
30,Bishopsgate,0.284403,E1 7HE,4,Coffee Shop,Pub,Cocktail Bar,Hotel,Gym / Fitness Center,Salad Place,...,Food Truck,Sushi Restaurant,Pizza Place,Breakfast Spot,Mediterranean Restaurant,Plaza,Street Food Gathering,Thai Restaurant,Vietnamese Restaurant,Clothing Store


### Grouping dataframe by Neighbourhood

In [83]:
city_of_london_merged_grouped = city_of_london_merged.groupby('Neighbourhood').mean().reset_index()
city_of_london_merged_grouped

,Neighbourhood,Latitude,Longitude,Distance to station,Cluster Labels
0,Aldersgate,51.518926,-0.096874,0.188473,0
1,Aldgate,51.513700,-0.080764,0.243603,3
2,Bassishaw,51.516894,-0.091467,0.250091,1
3,Billingsgate,51.510356,-0.083264,0.205363,3
4,Bishopsgate,51.517160,-0.078457,0.241461,4
5,Bread Street,51.514950,-0.098964,0.160436,2
6,Bridge,51.510610,-0.084770,0.099649,3
7,Broad Street,51.516769,-0.084480,0.183229,4
8,Candlewick,51.511130,-0.086263,0.067403,3
9,Castle Baynard,51.516898,-0.107419,0.310235,2


### Sorting newly created dataset by Distance to station in ascending way

In [84]:
city_of_london_merged_grouped_sorted = city_of_london_merged_grouped.sort_values(by='Distance to station', ascending=True)
city_of_london_merged_grouped_sorted

,Neighbourhood,Latitude,Longitude,Distance to station,Cluster Labels
15,Dowgate,51.511241,-0.090365,0.059741,1
8,Candlewick,51.511130,-0.086263,0.067403,3
21,Queenhithe,51.511548,-0.094901,0.077785,1
23,Vintry,51.512463,-0.095326,0.089757,1
6,Bridge,51.510610,-0.084770,0.099649,3
10,Cheap,51.515843,-0.097421,0.117323,2
22,Tower,51.511445,-0.074419,0.127459,3
5,Bread Street,51.514950,-0.098964,0.160436,2
18,Langbourn,51.512233,-0.081038,0.163050,3
24,Walbrook,51.514320,-0.087221,0.164188,1


### Grouping dataframe by Cluster Labels to check what would be the mean distance to station for every cluster

In [85]:
city_of_london_merged_cluster_grouped = city_of_london_merged_grouped_sorted.groupby('Cluster Labels').mean().reset_index()
city_of_london_merged_cluster_grouped

,Cluster Labels,Latitude,Longitude,Distance to station
0,0,51.520509,-0.094518,0.231368
1,1,51.513316,-0.091850,0.135434
2,2,51.516270,-0.101631,0.231584
3,3,51.511579,-0.081753,0.151088
4,4,51.515991,-0.081015,0.227661


### Sorting dataframe by Distance to station to check which cluster is the closest to the station

In [86]:
city_of_london_merged_cluster_grouped_sorted = city_of_london_merged_cluster_grouped.sort_values(by= 'Distance to station', ascending=True)
city_of_london_merged_cluster_grouped_sorted

,Cluster Labels,Latitude,Longitude,Distance to station
1,1,51.513316,-0.091850,0.135434
3,3,51.511579,-0.081753,0.151088
4,4,51.515991,-0.081015,0.227661
0,0,51.520509,-0.094518,0.231368
2,2,51.516270,-0.101631,0.231584


### Visualizing clustered neighbourhoods on map

In [88]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_of_london_merged['Latitude'], city_of_london_merged['Longitude'], city_of_london_merged['Neighbourhood'], city_of_london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Results

By analyzing and clustering dataset of City of London dataset, we were able to find out, that the best area for our project to build an office building will be within the first cluster and Ideally in Dowgate Neighbourhood.

In the discussions section we will sum up why it is the best option to begin our work with.


# Discussions

Let me explain you, why previously suggested solution is the best for our project. By examining our exploratory data analysis, we can see, that the first cluster is the most diverse cluster of all, having: coffee shops, restaurants, gyms, lounges, grocery stores and even a park, which is in fact really important, because study shows, that people working in a greener environment tend to be much productive than others, also in stressful situations they feel more relaxed and are able to make quicker decisions, which is extremely crucial in business related problems.

Furthermore, the first cluster is the closest to the station and has a wonderful public transport communication system, which is really important for people who does not drive a car.

Lastly this cluster has hotels within its sector included, which is highly appreciable when you are having international representatives of the company for a business convention.


# Conclusion

In the conclusion, to sum up my report, I would like to say, that by analyzing dataset of City of London, we were able to find out, that the best area to build an office building will be within first cluster and ideally in Dowgate neighbourhood. Because it has the most developed public transport communication system and there will be no hassle to get to work if you live outside of a Central London. Furthermore, it is the most progressive area of all, having everything you need (coffee shops, restaurants, gyms, lounges, grocery stores) and even more, to enjoy your everyday life at work.

##### Thank you very much for taking this journey with me, you have been wonderful!